#### Hasta este punto tenemos nuestras tablas procesadas (movies, ratings y tags) de forma correcta. El cliente nos ha solicitado apoyar al departamento de Marketing a realizar algunas consultas y a generar una única tabla y realizar algunos ajustes a la tabla final antes de poder almacenarla.

##### Nota: Para poder trabajar con este notebook es necesario haber terminado el ejercicio de la sesión 07

In [54]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
%run utils.py

In [55]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
from pyspark.sql import SparkSession, DataFrame, Column
import pyspark.sql.functions as f
import pyspark.sql.types as t

# Creación de sesión de Spark
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("ejercicio_8") \
    .getOrCreate()

# Carga de tablas requeridas
root_path = "../resources/data/tmp/parquet/"
names_list = ["07/movies", "07/ratings", "07/tags_p2"]
df_dict = read_tmp_df(spark, names_list)

movies_df = df_dict["07/movies"]
ratings_df = df_dict["07/ratings"]
tags_df = df_dict["07/tags_p2"]

movies_df.show(1, False)
ratings_df.show(1, False)
tags_df.show(1, False)

+--------+----------------+-------------------------------------------------+----+
|movie_id|title           |genres                                           |year|
+--------+----------------+-------------------------------------------------+----+
|1       |Toy Story (1995)|[Adventure, Animation, Children, Comedy, Fantasy]|1995|
+--------+----------------+-------------------------------------------------+----+
only showing top 1 row

+--------+----------+-------------+------------+-------------------+
|movie_id|avg_rating|stddev_rating|count_rating|min_time_rating    |
+--------+----------+-------------+------------+-------------------+
|1049    |3.4       |0.97         |5816        |1996-10-10 03:58:19|
+--------+----------+-------------+------------+-------------------+
only showing top 1 row

+--------+---------------------------------------------------------------------+-------------------+
|movie_id|tag_count                                                            |min_time_ta

#### En la siguiente imagen se muestra una representación a traves del diagrama de Venn sobre cada tabla (movies_df, ratings_df y tags_df) la cual fue contruida por el departamento de Marketing, el problema es que no saben la cantidad de datos de cada conjunto.
##### El cliente nos han solicitado obtener la cantidad de registros de cada conjunto de datos representado por las siguientes letras:
- ##### A: Registros de movies_df que no tiene filas coincidentes con las tablas ratings_df y tags_df
- ##### B: Registros de ratings_df que no tiene filas coincidentes con las tablas movies_df y tags_df
- ##### C: Registros de tags_df que no tiene filas coincidentes con las tablas movies_df y ratings_df
- ##### D: Registros de movies_df y ratings_df que no tiene filas coincidentes con la tabla tags_df
- ##### E: Registros de movies_df y tags_df que no tiene filas coincidentes con la tabla ratings_df
- ##### F: Registros de ratings_df y tags_df que no tiene filas coincidentes con la tabla movies_df
- ##### G: Registros que contiene datos coincidentes en las tablas movies_df, ratings_df y tags_df
-  Una tabla tiene registros coincidentes con otra cuando comparten el mismo valor en la columna "movie_id"

![title](../../resources/img/Tablas_conjuntos.png)

#### Actividad 1:
##### TO DO ->    Obtener los conjuntos de datos listados en el diagrama de Venn con operaciones de tipo join, con la finalidad de ahorrar recursos el cliente nos ha solicitado que en la salida de cada transformación el dataframe resultante contenga únicamente la columna "movie_id" (basta con utilizar únicante una transformación select al inicio de las operaciones join y utilizar únicamente joins del tipo **left_semi** y **left_anti**).
- ##### Los dataframes resultantes se almacenarán en las siguientes variables:
    - ##### A_df -> Conjunto A
    - ##### B_df -> Conjunto B
    - ##### C_df -> Conjunto C
    - ##### D_df -> Conjunto D
    - ##### E_df -> Conjunto E
    - ##### F_df -> Conjunto F
    - ##### G_df -> Conjunto G

In [56]:
# Método para conjuntos A, B y C:
def operacion_conjuntos_caso1(df1,df2,df3):
    df1_join_df2 = df1.select("movie_id").join(df2,["movie_id"],"left_semi")
    df1_join_df3 = df1.select("movie_id").join(df3,["movie_id"],"left_semi")
    resta_joins_df1jdf2_df1jdf3 = df1.select("movie_id")\
        .join(df1_join_df2,["movie_id"],"left_anti")\
        .join(df1_join_df3,["movie_id"],"left_anti")
    return resta_joins_df1jdf2_df1jdf3

In [57]:
# Conjunto A:
caso_a = operacion_conjuntos_caso1(movies_df,ratings_df,tags_df)
caso_a.show()
print(caso_a.count())

+--------+
|movie_id|
+--------+
+--------+

0


In [58]:
# Método para conjuntos D, E y F:
def operacion_conjuntos_caso2(df1,df2,df3):
    df1_join_df2 = df1.select("movie_id").join(df2,["movie_id"],"left_semi")
    resta_joins_df1jdf2 = df1_join_df2.select("movie_id").join(df3,["movie_id"],"left_anti")
    return resta_joins_df1jdf2

In [59]:
# Conjunto D:
caso_d = operacion_conjuntos_caso2(movies_df,ratings_df,tags_df)
caso_d.show(1)
print(caso_d.count())

+--------+
|movie_id|
+--------+
|      51|
+--------+
only showing top 1 row

28815


In [60]:
# Método para conjunto G:
def operacion_conjuntos_caso3(df1,df2,df3):
    df1_join_df2 = df1.select("movie_id").join(df2,["movie_id"],"left_semi")
    df1jdf2_join_df3 = df1_join_df2.select("movie_id").join(df3,["movie_id"],"left_semi")
    return df1jdf2_join_df3

In [61]:
# Conjunto G:
caso_g = operacion_conjuntos_caso3(movies_df,ratings_df,tags_df)
caso_g.show(1)
print(caso_g.count())

+--------+
|movie_id|
+--------+
|       1|
+--------+
only showing top 1 row

47903


In [62]:
## TU CODIGO VA EN ESTA CELDA:

# Conjunto A:
... # aplicar transformaciones join a movies_df, ratings_df y tags_df
A_df = operacion_conjuntos_caso1(movies_df,ratings_df,tags_df)
    
# Conjunto B:
... # aplicar transformaciones join a movies_df, ratings_df y tags_df
B_df = operacion_conjuntos_caso1(ratings_df,movies_df,tags_df)

# Conjunto C:
... # aplicar transformaciones join a movies_df, ratings_df y tags_df
C_df = operacion_conjuntos_caso1(tags_df,movies_df,ratings_df)

# Conjunto D:
... # aplicar transformaciones join a movies_df, ratings_df y tags_df
D_df = operacion_conjuntos_caso2(movies_df,ratings_df,tags_df)

# Conjunto E:
... # aplicar transformaciones join a movies_df, ratings_df y tags_df
E_df = operacion_conjuntos_caso2(movies_df,tags_df,ratings_df)

# Conjunto F:
... # aplicar transformaciones join a movies_df, ratings_df y tags_df
F_df = operacion_conjuntos_caso2(ratings_df,tags_df,movies_df)

# Conjunto G:
... # aplicar transformaciones join a movies_df, ratings_df y tags_df
G_df = operacion_conjuntos_caso3(movies_df,ratings_df,tags_df)


In [63]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
B_df.show(1)
"""
Ejemplo de salida de cada dataframe (desde A hasta G):
+--------+
|movie_id|
+--------+
|  179995|
+--------+
only showing top 1 row
"""

+--------+
|movie_id|
+--------+
|  179995|
+--------+
only showing top 1 row



'\nEjemplo de salida de cada dataframe (desde A hasta G):\n+--------+\n|movie_id|\n+--------+\n|  179995|\n+--------+\nonly showing top 1 row\n'

In [64]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA

assert A_df.count() == 0
assert B_df.count() == 4270
assert C_df.count() == 539
assert D_df.count() == 28815
assert E_df.count() == 2759
assert F_df.count() == 2251
assert G_df.count() == 47903

assert len(A_df.columns) == 1
assert len(B_df.columns) == 1
assert len(C_df.columns) == 1
assert len(D_df.columns) == 1
assert len(E_df.columns) == 1
assert len(F_df.columns) == 1
assert len(G_df.columns) == 1

assert "movie_id" in A_df.columns
assert "movie_id" in B_df.columns
assert "movie_id" in C_df.columns
assert "movie_id" in D_df.columns
assert "movie_id" in E_df.columns
assert "movie_id" in F_df.columns
assert "movie_id" in G_df.columns

#### Actividad 2:
##### TO DO ->    Algunos administradores de base de datos no comprenden el uso de los joins de tipo left_semi y left_anti, por lo tanto el cliente ha solicitado que tambien se realicen las transformaciones con cualquiera de los siguientes tipos de join: **left, right, inner, outer** (no utilizar left_semi o left_anti). Podrías utilizar la transformación filter/where en algunos casos.
##### No existe alguna reestricción de qué columnas contiene el dataframe de salida.
- ##### Los dataframes resultantes se almacenarán en las siguientes variables:
    - ##### A_df -> Conjunto A
    - ##### B_df -> Conjunto B
    - ##### C_df -> Conjunto C
    - ##### D_df -> Conjunto D
    - ##### E_df -> Conjunto E
    - ##### F_df -> Conjunto F
    - ##### G_df -> Conjunto G

In [65]:
# Operaciones de conjuntos pero sin usar left_semi ni left_anti:
# Caso A, B y C con join left:
def op_conj_1 (df1,df2,df3):
    primera_union = df1.join(df2,["movie_id"],"left")
    segunda_union = primera_union.join(df3,["movie_id"],"left")
    coincidencias_filtro = segunda_union.filter(((df2.movie_id.isNull()) & (df3.movie_id.isNull())))
    return coincidencias_filtro

In [66]:
# Caso A, B y C:
ej8_act2_caso_a = op_conj_1(movies_df,ratings_df,tags_df)
ej8_act2_caso_b = op_conj_1(ratings_df,movies_df,tags_df)
ej8_act2_caso_c = op_conj_1(tags_df, movies_df, ratings_df)
ej8_act2_caso_c.show(2)
print(ej8_act2_caso_c.count())
# A = 0, B = 4270, C = 539

+--------+--------------------+-------------------+-----+------+----+----------+-------------+------------+---------------+
|movie_id|           tag_count|       min_time_tag|title|genres|year|avg_rating|stddev_rating|count_rating|min_time_rating|
+--------+--------------------+-------------------+-----+------+----+----------+-------------+------------+---------------+
|  123693|[{ARMED CONFLICT,...|2018-05-17 04:35:32| null|  null|null|      null|         null|        null|           null|
|  123755|[{DRAMA, 1}, {MEL...|2018-05-17 04:36:12| null|  null|null|      null|         null|        null|           null|
+--------+--------------------+-------------------+-----+------+----+----------+-------------+------------+---------------+
only showing top 2 rows

539


In [67]:
movies_df.show(1)
ratings_df.show(1)
tags_df.show(1)

+--------+----------------+--------------------+----+
|movie_id|           title|              genres|year|
+--------+----------------+--------------------+----+
|       1|Toy Story (1995)|[Adventure, Anima...|1995|
+--------+----------------+--------------------+----+
only showing top 1 row

+--------+----------+-------------+------------+-------------------+
|movie_id|avg_rating|stddev_rating|count_rating|    min_time_rating|
+--------+----------+-------------+------------+-------------------+
|    1049|       3.4|         0.97|        5816|1996-10-10 03:58:19|
+--------+----------+-------------+------------+-------------------+
only showing top 1 row

+--------+--------------------+-------------------+
|movie_id|           tag_count|       min_time_tag|
+--------+--------------------+-------------------+
|  100001|[{PERFORMANCE (MO...|2018-05-18 01:40:21|
+--------+--------------------+-------------------+
only showing top 1 row



In [68]:
# Caso D, E y F:
def op_conj_2(df1,df2,df3):
    union_uno = df1.join(df3,["movie_id"],"left")
    union_dos = df2.join(df3,["movie_id"],"left")
    filtro_uno = union_uno.filter(df3.movie_id.isNull()).select(df1.columns)
    filtro_dos = union_dos.filter(df3.movie_id.isNull()).select(df2.columns)
    union_final = filtro_uno.join(filtro_dos,["movie_id"],"inner")
    return union_final

In [69]:
# Caso D:
ej8_act2_caso_d = op_conj_2(movies_df,ratings_df,tags_df)
#ej8_act2_caso_d.show(2)
print(ej8_act2_caso_d.count())
# Esperado 28815

28815


In [70]:
# Caso E:
caso_e_df = op_conj_2(movies_df,tags_df,ratings_df)
caso_e_df.show(2)
print(caso_e_df.count())
# Esperado = 2759

+--------+--------------------+--------------------+----+--------------------+-------------------+
|movie_id|               title|              genres|year|           tag_count|       min_time_tag|
+--------+--------------------+--------------------+----+--------------------+-------------------+
|  124037|The Secret Bride ...|             [Drama]|1934|[{MURDER, 1}, {PO...|2020-11-21 01:34:28|
|  138076|Diary of a Tired ...|[Comedy, Drama, R...|2009|[{INDEPENDENT FIL...|2018-05-17 04:54:58|
+--------+--------------------+--------------------+----+--------------------+-------------------+
only showing top 2 rows

2759


In [71]:
# Caso F:
caso_f_df = op_conj_2(ratings_df,tags_df,movies_df)
#caso_f_df.show(2)
print(caso_f_df.count())
# Esperado = 2251

2251


In [72]:
#Caso G - Registros que contiene datos coincidentes en las tablas movies_df, ratings_df y tags_df:
def op_conj_3(df1,df2,df3):
    union_uno = df1.join(df2,["movie_id"], "inner")
    union_dos = union_uno.join(df3,["movie_id"],"inner")
    return union_dos

In [73]:
# Caso G:
ej8_act2_caso_g = op_conj_3(movies_df,ratings_df,tags_df)
ej8_act2_caso_g.show(2)
print(ej8_act2_caso_g.count())
# Esperado = 47903

+--------+----------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+
|movie_id|           title|              genres|year|avg_rating|stddev_rating|count_rating|    min_time_rating|           tag_count|       min_time_tag|
+--------+----------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+
|       1|Toy Story (1995)|[Adventure, Anima...|1995|      3.89|         0.93|       76813|1996-01-28 18:00:00|[{1990S, 1}, {200...|2006-01-12 19:19:35|
|       2|  Jumanji (1995)|[Adventure, Child...|1995|      3.28|         0.96|       30209|1996-01-28 18:00:00|[{1860S, 1}, {196...|2006-01-13 14:14:30|
+--------+----------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+
only showing top 2 rows

47903


In [74]:
## TU CODIGO VA EN ESTA CELDA:

# Conjunto A:
A_df = op_conj_1(movies_df,ratings_df,tags_df)
# aplicar transformaciones join a movies_df, ratings_df y tags_df

# Conjunto B:
B_df = op_conj_1(ratings_df,movies_df,tags_df)
# aplicar transformaciones join a movies_df, ratings_df y tags_df

# Conjunto C:
C_df = op_conj_1(tags_df,movies_df,ratings_df)
# aplicar transformaciones join a movies_df, ratings_df y tags_df

# Conjunto D:
D_df = op_conj_2(movies_df,ratings_df,tags_df)
# aplicar transformaciones join a movies_df, ratings_df y tags_df

# Conjunto E:
E_df = op_conj_2(movies_df,tags_df,ratings_df)
# aplicar transformaciones join a movies_df, ratings_df y tags_df

# Conjunto F:
F_df = op_conj_2(ratings_df,tags_df,movies_df)
# aplicar transformaciones join a movies_df, ratings_df y tags_df

# Conjunto G:
G_df = op_conj_3(movies_df,ratings_df,tags_df)
# aplicar transformaciones join a movies_df, ratings_df y tags_df

In [75]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA

assert A_df.count() == 0
assert B_df.count() == 4270
assert C_df.count() == 539
assert D_df.count() == 28815
assert E_df.count() == 2759
assert F_df.count() == 2251
assert G_df.count() == 47903

#### Actividad 3:
##### TO DO ->    Con operaciones join genera un dataframe que contenga la union de todos los conjuntos (desde el A hasta el G) sin repetir registros. No utilices las transformaciones de union.

In [76]:
movies_df.show(2)
ratings_df.show(2)
tags_df.show(2)

+--------+----------------+--------------------+----+
|movie_id|           title|              genres|year|
+--------+----------------+--------------------+----+
|       1|Toy Story (1995)|[Adventure, Anima...|1995|
|       2|  Jumanji (1995)|[Adventure, Child...|1995|
+--------+----------------+--------------------+----+
only showing top 2 rows

+--------+----------+-------------+------------+-------------------+
|movie_id|avg_rating|stddev_rating|count_rating|    min_time_rating|
+--------+----------+-------------+------------+-------------------+
|    1049|       3.4|         0.97|        5816|1996-10-10 03:58:19|
|     339|      3.51|         0.93|       26845|1996-03-01 02:23:00|
+--------+----------+-------------+------------+-------------------+
only showing top 2 rows

+--------+--------------------+-------------------+
|movie_id|           tag_count|       min_time_tag|
+--------+--------------------+-------------------+
|  100001|[{PERFORMANCE (MO...|2018-05-18 01:40:21|
|  1

In [77]:
def join_universo(df1,df2,df3):
    join_df1df2 = df1.join(df2,["movie_id"],"outer")
    join_df1df2df3 = join_df1df2.join(df3,["movie_id"],"outer")
    return join_df1df2df3

In [78]:
# TEST:
TEST_val = join_universo(movies_df,ratings_df,tags_df)
TEST_val.show(1)
print(TEST_val.count())

+--------+--------------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+
|movie_id|               title|              genres|year|avg_rating|stddev_rating|count_rating|    min_time_rating|           tag_count|       min_time_tag|
+--------+--------------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+
|  100062|My Way (Mai Wei) ...|[Action, Drama, War]|2011|      3.63|         0.83|          64|2014-03-11 21:23:33|[{FATE, 1}, {PRES...|2018-05-26 17:40:54|
+--------+--------------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+
only showing top 1 row

86537


In [79]:
## TU CODIGO VA EN ESTA CELDA:
# aplicar transformaciones join a movies_df, ratings_df y tags_df
universe_df = join_universo(movies_df,ratings_df,tags_df)

In [80]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
universe_df.show(1)
"""
Ejemplo de salida:
+--------+--------------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+
|movie_id|               title|              genres|year|avg_rating|stddev_rating|count_rating|    min_time_rating|           tag_count|       min_time_tag|
+--------+--------------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+
|  100062|My Way (Mai Wei) ...|[Action, Drama, War]|2011|      3.63|         0.83|          64|2014-03-11 21:23:33|[{PRESS-GANGED, 1...|2018-05-26 16:40:54|
+--------+--------------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+
only showing top 1 row
"""

+--------+--------------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+
|movie_id|               title|              genres|year|avg_rating|stddev_rating|count_rating|    min_time_rating|           tag_count|       min_time_tag|
+--------+--------------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+
|  100062|My Way (Mai Wei) ...|[Action, Drama, War]|2011|      3.63|         0.83|          64|2014-03-11 21:23:33|[{FATE, 1}, {PRES...|2018-05-26 17:40:54|
+--------+--------------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+
only showing top 1 row



'\nEjemplo de salida:\n+--------+--------------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+\n|movie_id|               title|              genres|year|avg_rating|stddev_rating|count_rating|    min_time_rating|           tag_count|       min_time_tag|\n+--------+--------------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+\n|  100062|My Way (Mai Wei) ...|[Action, Drama, War]|2011|      3.63|         0.83|          64|2014-03-11 21:23:33|[{PRESS-GANGED, 1...|2018-05-26 16:40:54|\n+--------+--------------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+\nonly showing top 1 row\n'

In [81]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA

data = [Row(
    movie_id='100062', 
    title='My Way (Mai Wei) (2011)', 
    genres=['Action', 'Drama', 'War'], 
    year=2011, 
    avg_rating=3.63, 
    stddev_rating=0.83, 
    count_rating=64, 
    min_time_rating=datetime.datetime(2014, 3, 11, 21, 23, 33), 
    tag_count=[
        Row(tag='FATE', count=1),
        Row(tag='PRESS-GANGED', count=1),
        Row(tag='WAR', count=1),
        Row(tag='WORLD WAR II', count=1)],
    min_time_tag=datetime.datetime(2018, 5, 26, 16, 40, 54))]
schema = t.StructType([
    t.StructField("movie_id", t.StringType()),
    t.StructField("title", t.StringType()),
    t.StructField("genres", t.ArrayType(t.StringType())),
    t.StructField("year", t.IntegerType()),
    t.StructField("avg_rating", t.DoubleType()),
    t.StructField("stddev_rating", t.DoubleType()),
    t.StructField("count_rating", t.LongType()),
    t.StructField("min_time_rating", t.TimestampType()),
    t.StructField("tag_count", t.ArrayType(t.StructType([
        t.StructField("tag", t.StringType()),
        t.StructField("count", t.LongType())
    ]))),
    t.StructField("min_time_tag", t.TimestampType())
])
expected_df = spark.createDataFrame(data, schema)
expected_columns = ["movie_id", "title", "genres", "year",
                    "avg_rating", "stddev_rating", "count_rating",
                    "min_time_rating", "tag_count", "min_time_tag"]

assert universe_df.count() == 86537
assert set(universe_df.columns) - set(expected_columns) == set()
assert set(expected_columns) - set(universe_df.columns) == set()
assert universe_df \
    .select(*expected_columns) \
    .filter(f.col("movie_id") == "100062") \
    .subtract(expected_df).count() == 0

#### Actividad 4:
##### TO DO ->    La estructura del dataframe final de acuerdo al análisis realizado por marketing es el dado por la union del conjunto de datos: A, D, E y G. Realiza este proceso y almacena el dataframe resultante en la variable "final_df"
##### 

In [82]:
movies_df.show(1)
ratings_df.show(1)
tags_df.show(1)

+--------+----------------+--------------------+----+
|movie_id|           title|              genres|year|
+--------+----------------+--------------------+----+
|       1|Toy Story (1995)|[Adventure, Anima...|1995|
+--------+----------------+--------------------+----+
only showing top 1 row

+--------+----------+-------------+------------+-------------------+
|movie_id|avg_rating|stddev_rating|count_rating|    min_time_rating|
+--------+----------+-------------+------------+-------------------+
|    1049|       3.4|         0.97|        5816|1996-10-10 03:58:19|
+--------+----------+-------------+------------+-------------------+
only showing top 1 row

+--------+--------------------+-------------------+
|movie_id|           tag_count|       min_time_tag|
+--------+--------------------+-------------------+
|  100001|[{PERFORMANCE (MO...|2018-05-18 01:40:21|
+--------+--------------------+-------------------+
only showing top 1 row



In [83]:
# Definiciones
A_df = op_conj_1(movies_df,ratings_df,tags_df)
D_df = op_conj_2(movies_df,ratings_df,tags_df)
E_df = op_conj_2(movies_df,tags_df,ratings_df)
G_df = op_conj_3(movies_df,ratings_df,tags_df)

In [84]:
## TU CODIGO VA EN ESTA CELDA:
# aplicar transformaciones join a movies_df, ratings_df y tags_df
union_uno = A_df.unionByName(D_df, allowMissingColumns=True)
union_dos = union_uno.unionByName(E_df, allowMissingColumns=True)
union_tres = union_dos.unionByName(G_df, allowMissingColumns=True)
final_df = union_tres 

In [85]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
final_df.show(1)
"""
Ejemplo de salida:
+--------+----------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+
|movie_id|           title|              genres|year|avg_rating|stddev_rating|count_rating|    min_time_rating|           tag_count|       min_time_tag|
+--------+----------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+
|       1|Toy Story (1995)|[Adventure, Anima...|1995|      3.89|         0.93|       76813|1996-01-28 18:00:00|[{TIME TRAVEL, 11...|2006-01-12 19:19:35|
+--------+----------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+
only showing top 1 row
"""

+--------+--------------------+---------+----+----------+-------------+------------+-------------------+---------+------------+
|movie_id|               title|   genres|year|avg_rating|stddev_rating|count_rating|    min_time_rating|tag_count|min_time_tag|
+--------+--------------------+---------+----+----------+-------------+------------+-------------------+---------+------------+
|    1159|Love in Bloom (1935)|[Romance]|1935|      2.88|         1.05|          20|1996-10-11 05:28:21|     null|        null|
+--------+--------------------+---------+----+----------+-------------+------------+-------------------+---------+------------+
only showing top 1 row



'\nEjemplo de salida:\n+--------+----------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+\n|movie_id|           title|              genres|year|avg_rating|stddev_rating|count_rating|    min_time_rating|           tag_count|       min_time_tag|\n+--------+----------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+\n|       1|Toy Story (1995)|[Adventure, Anima...|1995|      3.89|         0.93|       76813|1996-01-28 18:00:00|[{TIME TRAVEL, 11...|2006-01-12 19:19:35|\n+--------+----------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+\nonly showing top 1 row\n'

In [86]:
final_df.where(f.col("movie_id") == 1).show()

+--------+----------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+
|movie_id|           title|              genres|year|avg_rating|stddev_rating|count_rating|    min_time_rating|           tag_count|       min_time_tag|
+--------+----------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+
|       1|Toy Story (1995)|[Adventure, Anima...|1995|      3.89|         0.93|       76813|1996-01-28 18:00:00|[{1990S, 1}, {200...|2006-01-12 19:19:35|
+--------+----------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+



In [87]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA

data = [Row(
    movie_id='100062', 
    title='My Way (Mai Wei) (2011)', 
    genres=['Action', 'Drama', 'War'], 
    year=2011, 
    avg_rating=3.63, 
    stddev_rating=0.83, 
    count_rating=64, 
    min_time_rating=datetime.datetime(2014, 3, 11, 21, 23, 33), 
    tag_count=[
        Row(tag='FATE', count=1),
        Row(tag='PRESS-GANGED', count=1),
        Row(tag='WAR', count=1),
        Row(tag='WORLD WAR II', count=1)],
    min_time_tag=datetime.datetime(2018, 5, 26, 16, 40, 54))]
schema = t.StructType([
    t.StructField("movie_id", t.StringType()),
    t.StructField("title", t.StringType()),
    t.StructField("genres", t.ArrayType(t.StringType())),
    t.StructField("year", t.IntegerType()),
    t.StructField("avg_rating", t.DoubleType()),
    t.StructField("stddev_rating", t.DoubleType()),
    t.StructField("count_rating", t.LongType()),
    t.StructField("min_time_rating", t.TimestampType()),
    t.StructField("tag_count", t.ArrayType(t.StructType([
        t.StructField("tag", t.StringType()),
        t.StructField("count", t.LongType())
    ]))),
    t.StructField("min_time_tag", t.TimestampType())
])
expected_df = spark.createDataFrame(data, schema)

expected_columns = ["movie_id", "title", "genres", "year",
                    "avg_rating", "stddev_rating", "count_rating",
                    "min_time_rating", "tag_count", "min_time_tag"]

assert final_df.count() == 79477

assert set(final_df.columns) - set(expected_columns) == set()
assert set(expected_columns) - set(final_df.columns) == set()
assert final_df \
    .select(*expected_columns) \
    .filter(f.col("movie_id") == "100062") \
    .subtract(expected_df).count() == 0

#### Actividad 5:
##### TO DO ->    El cliente nos ha solicitado llenar los valores "null" de la columna "year", para esto nos ha pedido seguir la siguiente regla:
- ##### Si la columna "year" es null y si la columna "min_time_rating" es null colocar el año de la columna "min_time_tag".
- ##### Si la columna "year" es null y si la columna "min_time_tag" es null colocar el año de la columna "min_time_rating".
- ##### Si la columna "year" es null, y las columnas "min_time_rating" y "min_time_tag" son distintas de null colocar el año menor de las columnas "min_time_rating" y "min_time_tag", en caso de que el año en ambas columnas sea el mismo colocar dicho año.
- ##### En cualquier otro caso mantener el valor entero 1970
##### Adicional nos ha solicitado generar una columna llamada "year_type" con los siguientes valores:
- ##### Si la columna "year" venia con un valor distinto a null, asignar el valor "YO"
- ##### Si la columna "year" es null y si la columna "min_time_rating" es null colocar "YT"
- ##### Si la columna "year" es null y si la columna "min_time_tag" es null colocar "YR"
- ##### Si la columna "year" es null, y las columnas "min_time_rating" y "min_time_tag" son distintas de null:
    - ##### Si "min_time_rating" es menor a "min_time_tag" colocar "YR"
    - ##### Si "min_time_tag" es menor a "min_time_rating" colocar "YT"
    - ##### Si "min_time_tag" es igual a "min_time_rating" colocar "YRT"
- ##### En cualquier otro caso mantener "YU"
##### Al finalizar este proceso eliminar las columnas "min_time_rating" y "min_time_tag"
- Para resolver estos ejercicios podrías utilizar la función year -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.year.html#pyspark.sql.functions.year
##### Este proceso se desarrollará en un método con la firma: def get_last_movies_df(df: DataFrame) -> DataFrame
##### La estructura del dataframe de salida será:
* |-- movie_id: string
* |-- title: string
* |-- avg_rating: double
* |-- count_rating: long
* |-- stddev_rating: double
* |-- genres: array
* |--- |-- element: string
* |-- tag_count: array
* |--- |-- element: struct
* |--- |--- |tag: string
* |--- |--- |count: long
* |-- year: integer
* |-- year_type: string
##### NO UTILIZAR withColumn NI withColumnRenamed, ni UDF

In [88]:
movies_df.show(1)
final_df.show(1)

+--------+----------------+--------------------+----+
|movie_id|           title|              genres|year|
+--------+----------------+--------------------+----+
|       1|Toy Story (1995)|[Adventure, Anima...|1995|
+--------+----------------+--------------------+----+
only showing top 1 row

+--------+--------------------+---------+----+----------+-------------+------------+-------------------+---------+------------+
|movie_id|               title|   genres|year|avg_rating|stddev_rating|count_rating|    min_time_rating|tag_count|min_time_tag|
+--------+--------------------+---------+----+----------+-------------+------------+-------------------+---------+------------+
|    1159|Love in Bloom (1935)|[Romance]|1935|      2.88|         1.05|          20|1996-10-11 05:28:21|     null|        null|
+--------+--------------------+---------+----+----------+-------------+------------+-------------------+---------+------------+
only showing top 1 row



In [89]:
# Condiciones
null_year = f.col("year").isNull()
null_mt_rat = f.col("min_time_rating").isNull()
null_mt_tag = f.col("min_time_tag").isNull()
not_null_mt_rat = f.col("min_time_rating").isNotNull()
not_null_mt_tag = f.col("min_time_tag").isNotNull()
año_mttag = f.year(f.col("min_time_tag"))
año_mtrat = f.year(f.col("min_time_rating"))

In [119]:
## TU CODIGO VA EN ESTA CELDA (NO UTILIZAR UDF):

#def get_last_movies_df(df: DataFrame) -> DataFrame:
    #return df # transformaciones a final_df

# Aplicar separación por conjuntos y luego union en df transformados:
def get_last_movies_df(df):
    year_nulo = df.filter(null_year)\
        .select(
            f.col("movie_id"),
            f.col("title"),
            f.col("avg_rating"),
            f.col("count_rating"),
            f.col("stddev_rating"),
            f.col("genres"),
            f.col("tag_count"),
            f.when(null_mt_rat & not_null_mt_tag, año_mttag)
                .when(null_mt_tag & not_null_mt_rat, año_mtrat)
                .when(not_null_mt_rat & not_null_mt_tag, f.least(f.year(f.col("min_time_rating")), f.year(f.col("min_time_tag"))))
                .otherwise(1970).alias("year"),
            f.when(((not_null_mt_rat & not_null_mt_tag) & (f.year(f.col("min_time_rating")) < f.year(f.col("min_time_tag")))) | null_mt_tag, 
                   f.lit("YR").alias("year_type"))
                .when(((not_null_mt_rat & not_null_mt_tag) & (f.year(f.col("min_time_tag")) < f.year(f.col("min_time_rating")))) | null_mt_rat, 
                      f.lit("YT").alias("year_type"))
                .when(f.year(f.col("min_time_rating")) == f.year(f.col("min_time_tag")), f.lit("YRT").alias("year_type"))
                .otherwise(f.lit("YU")).alias("year_type")
        ).drop("min_time_rating", "min_time_tag")
    
    year_no_nulo = df.filter(f.col("year").isNotNull())\
        .select(
            f.col("movie_id"),
            f.col("title"),
            f.col("avg_rating"),
            f.col("count_rating"),
            f.col("stddev_rating"),
            f.col("genres"),
            f.col("tag_count"),
            f.col("year"),
            f.when(f.col("year").isNotNull(), f.lit("YO").alias("year_type"))
        ).drop("min_time_rating", "min_time_tag")

    df = year_nulo.union(year_no_nulo)
    return df
    

In [120]:
#TEST
movies_df = get_last_movies_df(final_df)
movies_df.where(f.col("movie_id") == 1).show(1,False)
#movies_df.filter(f.col("year").isNull()).show() #demostrar que ya no hay null
#movies_df.filter(f.col("year").isNotNull()).show(3)

+--------+----------------+----------+------------+-------------+-------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [122]:
print(movies_df.where(f.col("year_type") == "YO").count())
print(movies_df.where(f.col("year_type") == "YR").count())
print(movies_df.where(f.col("year_type") == "YRT").count())
print(movies_df.where(f.col("year_type") == "YT").count())
# YO = 79235, YR = 159, YRT = 55, YT= 28

79235
159
55
28


In [123]:
movies_df.printSchema()

root
 |-- movie_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- avg_rating: double (nullable = true)
 |-- count_rating: long (nullable = true)
 |-- stddev_rating: double (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- tag_count: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- tag: string (nullable = true)
 |    |    |-- count: long (nullable = true)
 |-- year: integer (nullable = true)
 |-- year_type: string (nullable = true)



In [124]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA

movies_df = get_last_movies_df(final_df)
movies_df.show(1)
"""
Ejemplo de salida esperada (el orden de la columnas podría ser distinto):
+--------+----------------+----------+------------+-------------+--------------------+--------------------+----+---------+
|movie_id|           title|avg_rating|count_rating|stddev_rating|              genres|           tag_count|year|year_type|
+--------+----------------+----------+------------+-------------+--------------------+--------------------+----+---------+
|       1|Toy Story (1995)|      3.89|       76813|         0.93|[Adventure, Anima...|[{1990S, 1}, {200...|1995|       YO|
+--------+----------------+----------+------------+-------------+--------------------+--------------------+----+---------+
only showing top 1 row
"""

+--------+------------+----------+------------+-------------+--------------------+---------+----+---------+
|movie_id|       title|avg_rating|count_rating|stddev_rating|              genres|tag_count|year|year_type|
+--------+------------+----------+------------+-------------+--------------------+---------+----+---------+
|  148093|The Republic|      2.33|           3|         0.62|[Action, Crime, T...|     null|2015|       YR|
+--------+------------+----------+------------+-------------+--------------------+---------+----+---------+
only showing top 1 row



'\nEjemplo de salida esperada (el orden de la columnas podría ser distinto):\n+--------+----------------+----------+------------+-------------+--------------------+--------------------+----+---------+\n|movie_id|           title|avg_rating|count_rating|stddev_rating|              genres|           tag_count|year|year_type|\n+--------+----------------+----------+------------+-------------+--------------------+--------------------+----+---------+\n|       1|Toy Story (1995)|      3.89|       76813|         0.93|[Adventure, Anima...|[{1990S, 1}, {200...|1995|       YO|\n+--------+----------------+----------+------------+-------------+--------------------+--------------------+----+---------+\nonly showing top 1 row\n'

In [125]:
movies_df.where(f.col("movie_id") == 179479).show(1,False)

+--------+----------------------------------------------+----------+------------+-------------+-------------+-------------------------------------------------------------------------------------------+----+---------+
|movie_id|title                                         |avg_rating|count_rating|stddev_rating|genres       |tag_count                                                                                  |year|year_type|
+--------+----------------------------------------------+----------+------------+-------------+-------------+-------------------------------------------------------------------------------------------+----+---------+
|179479  |Samadhi Part 1: Maya, the Illusion of the Self|4.19      |8           |0.75         |[Documentary]|[{EASTERN PHILOSOPHY, 1}, {MEDITATION, 1}, {METAPHYSICAL, 1}, {NEW AGE, 1}, {SPIRITUAL, 1}]|2017|YT       |
+--------+----------------------------------------------+----------+------------+-------------+-------------+-----------------------

In [126]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA

test_value = [Row(
    movie_id='179479',
    title='Samadhi Part 1: Maya, the Illusion of the Self',
    avg_rating=4.19,
    count_rating=8,
    stddev_rating=0.75,
    genres=['Documentary'],
    tag_count=[Row(tag='EASTERN PHILOSOPHY', count=1),
               Row(tag='MEDITATION', count=1),
               Row(tag='METAPHYSICAL', count=1),
               Row(tag='NEW AGE', count=1),
               Row(tag='SPIRITUAL', count=1)],
    year=2017,
    year_type='YT')]

expected_count_by_year_type = [
    Row(year_type='YO', count=79235),
    Row(year_type='YR', count=159),
    Row(year_type='YRT', count=55),
    Row(year_type='YT', count=28)]

expected_columns = ["movie_id","title","avg_rating","count_rating","stddev_rating","genres","tag_count","year","year_type"]

expected_schema = 'movie_id STRING,title STRING,avg_rating DOUBLE,count_rating BIGINT,stddev_rating DOUBLE,genres ARRAY<STRING>,tag_count ARRAY<STRUCT<tag: STRING, count: BIGINT>>,year INT,year_type STRING'

assert set(movies_df.columns) - set(expected_columns) == set()
assert set(expected_columns) - set(movies_df.columns) == set()
assert movies_df.filter(f.col("year").isNull()).count() == 0
assert movies_df \
    .select(*expected_columns) \
    .filter(f.col("movie_id") == "179479") \
    .collect() == test_value
assert schema_to_ddl(spark, movies_df.select(*expected_columns)) == expected_schema
assert movies_df.groupBy(f.col("year_type")).count().orderBy(f.col("count").desc()).collect() == expected_count_by_year_type

In [127]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
dfs = [(movies_df, "08/movies")]

write_tmp_df(dfs)